This notebook demonstrates how to stream OPERA S1-RTC data directly from the ASF DAAC. It requires login credentials in a .netrc file

[Hurricane Otis](https://en.wikipedia.org/wiki/Hurricane_Otis) made landfall in October 2023 near Acapulco, Mexico as a category 5 hurricane. We will query ASF DAAC to retrieve OPERA S1-RTC data for this period.

In [70]:
from datetime import datetime
import asf_search as asf
from shapely.geometry import Point
import rasterio

from osgeo import gdal

In [71]:
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

In [57]:
start_date = datetime(year=2023, month=10, day=25)
start_date.strftime("%Y-%m-%dT00:00:00Z")

'2023-10-25T00:00:00Z'

In [64]:
# Search dates
start_date = datetime(year=2023, month=10, day=25)
end_date = datetime(year=2023, month=11, day=8)

# AOI
aoi_wkt = 'POLYGON((-100.0218 16.7927,-99.7753 16.7927,-99.7753 16.9841,-100.0218 16.9841,-100.0218 16.7927))'

opts = {
    'intersectsWith':Point(-99.8796, 16.8421).wkt,
    'platform' : asf.PLATFORM.SENTINEL1,
    'start': start_date.strftime("%Y-%m-%dT00:00:00Z"),
    'end': end_date.strftime("%Y-%m-%dT00:00:00Z"),
    'polarization': ["VV", "VH"],
    'maxResults' : 250,
    'processingLevel' : 'RTC', 
    'beamMode' : asf.BEAMMODE.IW,
    # 'flightDirection' : asf.FLIGHT_DIRECTION.ASCENDING
}

In [65]:
results = asf.search(**opts)
len(results)

4

In [66]:
results[1].geojson()

{'type': 'Feature',
 'geometry': {'coordinates': [[[-99.92892, 16.73507],
    [-99.06057, 16.89631],
    [-99.09463, 17.06264],
    [-99.959, 16.90243],
    [-99.92892, 16.73507]]],
  'type': 'Polygon'},
 'properties': {'beamModeType': 'IW',
  'browse': ['https://datapool.asf.alaska.edu/BROWSE/OPERA-S1/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0_BROWSE.png',
   's3://asf-cumulus-prod-opera-browse/OPERA_L2_RTC-S1/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0_BROWSE.png',
   'https://datapool.asf.alaska.edu/BROWSE/OPERA-S1/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0_BROWSE_low-res.png',
   's3://asf-cumulus-prod-opera-browse/OPERA_L2_RTC-S1/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0_BROWSE_low-res.png'

{'type': 'Feature',
 'geometry': {'coordinates': [[[-53.2003, 55.6359],
    [-57.081, 55.3225],
    [-56.4286, 53.4867],
    [-52.7191, 53.7917],
    [-53.2003, 55.6359]]],
  'type': 'Polygon'},
 'properties': {'beamModeType': 'IW',
  'browse': None,
  'bytes': 1644202945,
  'centerLat': 54.5744,
  'centerLon': -54.8597,
  'faradayRotation': None,
  'fileID': 'S1A_IW_RAW__0SDV_20231106T212447_20231106T212518_051102_0629A9_9065-RAW',
  'flightDirection': 'ASCENDING',
  'groupID': 'S1A_IWDV_0175_0181_051102_105',
  'granuleType': 'SENTINEL_1A_FRAME',
  'insarStackId': None,
  'md5sum': '272d8042cc0750b47a0b2d26d1a0cf7f',
  'offNadirAngle': None,
  'orbit': 51102,
  'pathNumber': 105,
  'platform': 'Sentinel-1A',
  'pointingAngle': None,
  'polarization': 'VV+VH',
  'processingDate': '2023-11-06T21:24:47.965Z',
  'processingLevel': 'RAW',
  'sceneName': 'S1A_IW_RAW__0SDV_20231106T212447_20231106T212518_051102_0629A9_9065',
  'sensor': 'C-SAR',
  'startTime': '2023-11-06T21:24:47.965Z',
  

In [23]:
import asf_search as asf
options = {
	'intersectsWith': 'POLYGON((-100.0218 16.7927,-99.7753 16.7927,-99.7753 16.9841,-100.0218 16.9841,-100.0218 16.7927))',
	'collections': [
		'C1259974840-ASF',
		'C1259976861-ASF',
		'C1259981910-ASF',
		'C1259982010-ASF',
		'C1259975087-ASFDEV',
		'C1259976862-ASFDEV',
		'C1259983643-ASFDEV',
		'C1259983645-ASFDEV',
		'C2777443834-ASF',
		'C2777436413-ASF'
	],
	'start': '2023-10-23T07:00:00Z',
	'maxResults': 250
}
results = asf.search(**options)

In [72]:
with rasterio.open('https://datapool.asf.alaska.edu/RTC/OPERA-S1/OPERA_L2_RTC-S1_T078-165486-IW2_20231105T004829Z_20231106T002602Z_S1A_30_v1.0_VH.tif') as ds:
    print(ds.profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 3430, 'height': 1592, 'count': 1, 'crs': CRS.from_epsg(32614), 'transform': Affine(30.0, 0.0, 391830.0,
       0.0, -30.0, 1893570.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}
